In [1]:
import cv2
import os
import random
from glob import glob

# Veri seti klasörünü ve çıktı klasörünü belirtin
input_folder = "/kaggle/input/cataract-dr-normal-glaucoma-fundus-images-dataset/split/train/cataract/"  # Fundus görüntülerinizin bulunduğu klasör
output_folder = "/kaggle/working/Cataract"  # İşlenmiş görüntülerin kaydedileceği klasör
os.makedirs(output_folder, exist_ok=True)

# Hedef boyut
target_size = (512, 512)

# Görselleri işleme fonksiyonu
def process_image(image_path, output_path):
    # Görseli yükle
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # 512x512'lik bir kırpma bölgesini merkeze yerleştir
    crop_x_start = (image.shape[1]) // 2  # Merkezden başlamak için sol kenarın X koordinatı
    crop_y_start = (image.shape[0]) // 2  # Merkezden başlamak için üst kenarın Y koordinatı
    crop_x_start = crop_x_start - crop_y_start
    # Görseli kırp
    cropped_image = image[0:0+image.shape[0], crop_x_start:crop_x_start+image.shape[0]]
    
    # CLAHE uygulaması için LAB renk uzayına dönüştür
    lab_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    
    # CLAHE'yi uygulayın
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)
    
    # LAB kanallarını birleştir
    processed_lab = cv2.merge((cl, a_channel, b_channel))
    
    # Görseli tekrar BGR uzayına dönüştür
    processed_image = cv2.cvtColor(processed_lab, cv2.COLOR_LAB2BGR)

    # Görseli 512x512 olarak yeniden boyutlandır
    resized_image = cv2.resize(processed_image, (512, 512))
    
    # İşlenmiş görseli kaydet
    cv2.imwrite(output_path, resized_image)

# Klasördeki tüm görüntüleri al
image_paths = glob(os.path.join(input_folder, "*.jpg"))  # Tüm .png dosyalarını al

# Kullanıcıdan seçim sayısını iste
num_images_to_process = int(input(f"Klasördeki {len(image_paths)} görüntüden kaç tanesini işlemek istiyorsunuz? "))

# Kullanıcıdan rastgele mi yoksa sıralı mı seçim yapacağını sor
selection_type = input("Rastgele seçim yapmak istiyorsanız 'r', sıralı seçim için 's' yazın: ").strip().lower()

# Görüntüleri seç
if selection_type == 'r':
    selected_image_paths = random.sample(image_paths, min(num_images_to_process, len(image_paths)))
elif selection_type == 's':
    selected_image_paths = image_paths[:min(num_images_to_process, len(image_paths))]
else:
    print("Geçersiz seçim türü! Varsayılan olarak sıralı seçim yapılacak.")
    selected_image_paths = image_paths[:min(num_images_to_process, len(image_paths))]

# Seçilen görüntüleri işleme
for image_path in selected_image_paths:
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, filename)
    process_image(image_path, output_path)

print(f"İşlem tamamlandı! İşlenmiş görüntüler {output_folder} klasörüne kaydedildi.")


Klasördeki 655 görüntüden kaç tanesini işlemek istiyorsunuz?  400
Rastgele seçim yapmak istiyorsanız 'r', sıralı seçim için 's' yazın:  r


İşlem tamamlandı! İşlenmiş görüntüler /kaggle/working/Cataract klasörüne kaydedildi.


In [2]:
import torch
import cv2
import os
import numpy as np
from PIL import Image
import functools
import torch.nn as nn
import torch.nn.functional as F

# Residual Dense Block (RDB)
class ResidualDenseBlock_5C(nn.Module):
    def __init__(self, nf=64, gc=32, bias=True):
        super(ResidualDenseBlock_5C, self).__init__()
        self.conv1 = nn.Conv2d(nf, gc, 3, 1, 1, bias=bias)
        self.conv2 = nn.Conv2d(nf + gc, gc, 3, 1, 1, bias=bias)
        self.conv3 = nn.Conv2d(nf + 2 * gc, gc, 3, 1, 1, bias=bias)
        self.conv4 = nn.Conv2d(nf + 3 * gc, gc, 3, 1, 1, bias=bias)
        self.conv5 = nn.Conv2d(nf + 4 * gc, nf, 3, 1, 1, bias=bias)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return x5 * 0.2 + x

# RRDB bloğu
class RRDB(nn.Module):
    def __init__(self, nf, gc=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nf, gc)
        self.RDB2 = ResidualDenseBlock_5C(nf, gc)
        self.RDB3 = ResidualDenseBlock_5C(nf, gc)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out * 0.2 + x

# RRDB katmanları oluşturma
def make_layer(block, n_layers):
    layers = []
    for _ in range(n_layers):
        layers.append(block())
    return nn.Sequential(*layers)

# ESRGAN model mimarisi (RRDBNet)
class RRDBNet(nn.Module):
    def __init__(self, in_nc=3, out_nc=3, nf=64, nb=23, gc=32):
        super(RRDBNet, self).__init__()
        RRDB_block_f = functools.partial(RRDB, nf=nf, gc=gc)

        self.conv_first = nn.Conv2d(in_nc, nf, 3, 1, 1, bias=True)
        self.RRDB_trunk = make_layer(RRDB_block_f, nb)
        self.trunk_conv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        #### upsampling
        self.upconv1 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(nf, nf, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(nf, out_nc, 3, 1, 1, bias=True)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk

        fea = self.lrelu(self.upconv1(F.interpolate(fea, scale_factor=2, mode='nearest')))
        fea = self.lrelu(self.upconv2(F.interpolate(fea, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.HRconv(fea)))

        return out


# ESRGAN modelini yükleme
def load_esrgan_model(model_path, device):
    model = RRDBNet(in_nc=3, out_nc=3, nf=64, nb=23)  # Model mimarisini oluştur
    state_dict = torch.load(model_path, map_location=device, weights_only=True)  # Ağırlıkları yükle
    model.load_state_dict(state_dict)  # Ağırlıkları modele yükle
    model.to(device)  # Modeli belirtilen cihaza taşı
    model.eval()  # Modeli eval moduna al
    return model


# Görüntüyü yükleme ve ön işleme
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = np.array(img)
    img = img.astype(np.float32) / 255.0  # Normalize et
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)  # PyTorch tensor formatına çevir
    return img

# Görüntüyü ESRGAN ile iyileştirme
def enhance_image_with_esrgan(model, image_tensor):
    with torch.no_grad():
        output = model(image_tensor)
    return output.squeeze(0).permute(1, 2, 0).clamp(0, 1).cpu().numpy()  # CPU'ya taşındı


# Görüntüyü kaydetme
def save_image(image_numpy, output_path):
    image_numpy = (image_numpy * 255.0).astype(np.uint8)
    image = Image.fromarray(image_numpy)
    image.save(output_path)

# Tüm klasörleri işleme
def process_dataset(input_root, output_root, model, device):
    os.makedirs(output_root, exist_ok=True)

    for root, dirs, files in os.walk(input_root):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                input_image_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_root)
                output_folder = os.path.join(output_root, relative_path)
                os.makedirs(output_folder, exist_ok=True)
                output_image_path = os.path.join(output_folder, file)

                # Görüntüyü yükle ve tensor formatına çevir
                image_tensor = preprocess_image(input_image_path).to(device)  # Doğrudan device kullanıldı
                enhanced_image = enhance_image_with_esrgan(model, image_tensor)
                save_image(enhanced_image, output_image_path)
                print(f"Processed: {input_image_path} -> {output_image_path}")



# Örnek kullanım
def main():
    # GPU kullanılabilir mi kontrol et
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Kullanılan cihaz: {device}")

    model_path = '/kaggle/input/rrdb_esrgan_x4.pth/pytorch/default/1/RRDB_ESRGAN_x4.pth'  # Modelin konumu
    input_root = '/kaggle/working/Cataract'
    output_root = '/kaggle/working/Cataract_output_dataset'
    
    model = load_esrgan_model(model_path, device)
    process_dataset(input_root, output_root, model, device)

if __name__ == '__main__':
    main()


Kullanılan cihaz: cuda
Processed: /kaggle/working/Cataract/_14_7805520.jpg -> /kaggle/working/Cataract_output_dataset/./_14_7805520.jpg
Processed: /kaggle/working/Cataract/_159_1399279.jpg -> /kaggle/working/Cataract_output_dataset/./_159_1399279.jpg
Processed: /kaggle/working/Cataract/2216_right.jpg -> /kaggle/working/Cataract_output_dataset/./2216_right.jpg
Processed: /kaggle/working/Cataract/345_right.jpg -> /kaggle/working/Cataract_output_dataset/./345_right.jpg
Processed: /kaggle/working/Cataract/_327_3715152.jpg -> /kaggle/working/Cataract_output_dataset/./_327_3715152.jpg
Processed: /kaggle/working/Cataract/_400_7591244.jpg -> /kaggle/working/Cataract_output_dataset/./_400_7591244.jpg
Processed: /kaggle/working/Cataract/_339_9106724.jpg -> /kaggle/working/Cataract_output_dataset/./_339_9106724.jpg
Processed: /kaggle/working/Cataract/103_left.jpg -> /kaggle/working/Cataract_output_dataset/./103_left.jpg
Processed: /kaggle/working/Cataract/2199_left.jpg -> /kaggle/working/Cataract

In [3]:
import shutil

destination_root='/kaggle/working/Cataract_output_dataset'

# Hedef ZIP dosyasının adı ve konumu
zip_file_name = "/kaggle/working/Cataract_output_dataset.zip"

# Hedef klasörü ZIP'e sıkıştır
shutil.make_archive(base_name=zip_file_name.replace('.zip', ''), 
                    format='zip', 
                    root_dir=destination_root)

print(f"ZIP file created at: {zip_file_name}")

ZIP file created at: /kaggle/working/Cataract_output_dataset.zip
